## Recordemos lo que es una base de datos

Es un almacén o bóveda en donde el sistema guarda la información seleccionada por el usuario. Dicha información puede ser almacenada utilizando estructuras definidas, de manera que acceder a la información sea un proceso rápido dentro de la base de datos.

<center><img src="https://miro.medium.com/v2/resize:fit:750/0*rwqI3XKmDZgsVuxr.jpg"/></center>


### Tipos de bases de datos

<center><img SRC="https://gowithcode.com/wp-content/uploads/2021/04/sql-vs-nosql.jpg"/></center>

### OJO: No es lo mismo una base de datos a un gestor de base de datos

* "Yo manejo bases de datos SQLsever"
* "Yo sé oracle"
* "Yo manejo bases de datos MongoDB"
* "Yo sé MySQL"

#### Administrador de base de datos (Motor de base de datos)

<center><img src="https://i0.wp.com/www.diarlu.com/wp-content/uploads/2019/02/gestores_bases-datos.jpg?fit=1000%2C600&ssl=1" /></center>



## Diagrama ER (Entidad relación)

El diagrama entidad relación nos permite tener una visión del relacionamiento y la cantidad de tablas que se utilizan para una aplicación.

<center><img src="https://mandalay.mx/wp-content/uploads/2023/05/Structure-of-the-relational-database-Mvsemdm-Each-box-on-the-figure-above-contains-one.png"/></center>









## Como llevamos esa información a nuestro desarrollo ?

La información de las bases de datos se pueden obtener mediante un **conector**

Los conectores de bases de datos nos permiten conectarnos a la base de datos, estos transforman una cadena de texto con una instrucción (**Consulta**) para ser ejecutada en la base de datos y traer la información requerida.

### *Usemos SQL_lite* como base de datos

DIAGRAMA ENTIDAD RELACION

In [86]:
!ls -la

total 16
drwxr-xr-x 1 root root 4096 Sep  1 16:44 .
drwxr-xr-x 1 root root 4096 Sep  1 12:56 ..
drwxr-xr-x 4 root root 4096 Aug 30 13:24 .config
drwxr-xr-x 1 root root 4096 Aug 30 13:25 sample_data


In [128]:
import sqlite3

connection = sqlite3.connect("MiTiendaCom.db")

In [129]:
!ls -la

total 16
drwxr-xr-x 1 root root 4096 Sep  1 17:56 .
drwxr-xr-x 1 root root 4096 Sep  1 12:56 ..
drwxr-xr-x 4 root root 4096 Aug 30 13:24 .config
-rw-r--r-- 1 root root    0 Sep  1 17:56 MiTiendaCom.db
drwxr-xr-x 1 root root 4096 Aug 30 13:25 sample_data


In [130]:
# Creamos un cursor (instanciamos el objeto cursor)
c = connection.cursor()

In [131]:
c.execute("""
  -- Crear la tabla de Clientes
  CREATE TABLE Clientes (
    ID_Cliente INTEGER PRIMARY KEY AUTOINCREMENT,
    Nombre TEXT NOT NULL,
    Apellido TEXT NOT NULL,
    Email TEXT UNIQUE,
    Telefono TEXT
);
""")
# Confirmar la ejecución
connection.commit()

In [132]:
!ls -la

total 32
drwxr-xr-x 1 root root  4096 Sep  1 17:56 .
drwxr-xr-x 1 root root  4096 Sep  1 12:56 ..
drwxr-xr-x 4 root root  4096 Aug 30 13:24 .config
-rw-r--r-- 1 root root 16384 Sep  1 17:56 MiTiendaCom.db
drwxr-xr-x 1 root root  4096 Aug 30 13:25 sample_data


In [133]:
from logging import exception
## Vamos a generar una función para terminar todo.

def get_connection():
  return sqlite3.connect("MiTiendaCom.db")

def exec_instruction(cmd: str):
  try:
    conn = get_connection()
    c = conn.cursor()
    c.execute(cmd)
    # Confirmar la ejecución
    conn.commit()
    conn.close()
    print("Executed Succesfully")
  except:
    print("Error in query execution")



In [134]:
tabla1 = """-- Crear la tabla de Categorías
CREATE TABLE Categorias (
    ID_Categoria INTEGER PRIMARY KEY AUTOINCREMENT,
    Nombre TEXT NOT NULL UNIQUE
);"""

tabla2 = """-- Crear la tabla de Productos
CREATE TABLE Productos (
    ID_Producto INTEGER PRIMARY KEY AUTOINCREMENT,
    Nombre TEXT NOT NULL,
    Precio REAL NOT NULL,
    Cantidad INTEGER NOT NULL,
    ID_Categoria INTEGER,
    FOREIGN KEY (ID_Categoria) REFERENCES Categorias(ID_Categoria)
);"""

In [135]:
exec_instruction(tabla1)
exec_instruction(tabla2)

Executed Succesfully
Executed Succesfully


In [136]:
rest = """
  -- Crear la tabla de Empleados
  CREATE TABLE Empleados (
      ID_Empleado INTEGER PRIMARY KEY AUTOINCREMENT,
      Nombre TEXT NOT NULL,
      Apellido TEXT NOT NULL,
      Cargo TEXT NOT NULL
  );
"""
rest1="""
  -- Crear la tabla de Ventas
  CREATE TABLE Ventas (
      ID_Venta INTEGER PRIMARY KEY AUTOINCREMENT,
      Fecha DATE NOT NULL,
      ID_Cliente INTEGER,
      ID_Empleado INTEGER,
      FOREIGN KEY (ID_Cliente) REFERENCES Clientes(ID_Cliente),
      FOREIGN KEY (ID_Empleado) REFERENCES Empleados(ID_Empleado)
  );
"""
rest2 = """
  -- Crear la tabla de DetallesVenta
  CREATE TABLE DetallesVenta (
      ID_Detalle INTEGER PRIMARY KEY AUTOINCREMENT,
      ID_Venta INTEGER,
      ID_Producto INTEGER,
      Cantidad INTEGER NOT NULL,
      Precio_Unitario REAL NOT NULL,
      FOREIGN KEY (ID_Venta) REFERENCES Ventas(ID_Venta),
      FOREIGN KEY (ID_Producto) REFERENCES Productos(ID_Producto)
  );
"""
exec_instruction(rest)
exec_instruction(rest1)
exec_instruction(rest2)


Executed Succesfully
Executed Succesfully
Executed Succesfully


## Como insertar data? ... Con comando SQL

Ojo ver el poco control que tenemos sobre la sintaxis



In [137]:
insert = """
  INSERT INTO Clientes (Nombre, Apellido, Email, Telefono) VALUES
('Juan', 'Pérez', 'juan.perez@email.com', '1234567890'),
"""
exec_instruction(insert)

Error in query execution


In [138]:
insert = """
  INSERT INTO Clientes (Nombre, Apellido, Email, Telefono) VALUES
('Juan', 'Pérez', 'juan.perez@email.com', '1234567890')
"""
exec_instruction(insert)

Executed Succesfully


## Vamos a automatizar la populada de los datos

In [139]:
# Usemos el execute many

# c.executemany(Query,list_values)
def exec_instruction(cmd: str, many:bool=False,list_rec=[]):
  try:
    conn = get_connection()
    c = conn.cursor()
    if many:
      c.executemany(cmd,list_rec)
    else:
      c.execute(cmd)
    # Confirmar la ejecución
    conn.commit()
    conn.close()
    print("Executed Succesfully")
  except:
    print("Error in query execution")


In [140]:
# @title Datos de clientes ... (clientes)

clientes = [
    ('Ana', 'García', 'ana.garcia@email.com', '0987654321'),
    ('Luis', 'Martínez', 'luis.martinez@email.com', '1122334455'),
    ('Sofía', 'Rodríguez', 'sofia.rodriguez@email.com', '2233445566'),
    ('Carlos', 'López', 'carlos.lopez@email.com', '3344556677'),
    ('María', 'González', 'maria.gonzalez@email.com', '4455667788'),
    ('David', 'Fernández', 'david.fernandez@email.com', '5566778899'),
    ('Laura', 'Ruiz', 'laura.ruiz@email.com', '6677889900'),
    ('Daniel', 'Ramírez', 'daniel.ramirez@email.com', '7788990011'),
    ('Elena', 'Morales', 'elena.morales@email.com', '8899001122'),
    ('Javier', 'Ortega', 'javier.ortega@email.com', '9900112233'),
    ('Sara', 'Gutiérrez', 'sara.gutierrez@email.com', '0011223344'),
    ('Pablo', 'Romero', 'pablo.romero@email.com', '1122334455'),
    ('Marta', 'Sánchez', 'marta.sanchez@email.com', '2233445566'),
    ('José', 'Torres', 'jose.torres@email.com', '3344556677'),
    ('Andrea', 'Giménez', 'andrea.gimenez@email.com', '4455667788'),
    ('Jorge', 'Vázquez', 'jorge.vazquez@email.com', '5566778899'),
    ('Mónica', 'Castro', 'monica.castro@email.com', '6677889900'),
    ('Diego', 'Serrano', 'diego.serrano@email.com', '7788990011'),
    ('Cristina', 'Guerrero', 'cristina.guerrero@email.com', '8899001122'),
    ('Rubén', 'Molina', 'ruben.molina@email.com', '9900112233'),
    ('Isabel', 'Delgado', 'isabel.delgado@email.com', '0011223344'),
    ('Álvaro', 'Marín', 'alvaro.marin@email.com', '1122334455'),
    ('Irene', 'Núñez', 'irene.nunez@email.com', '2233445566'),
    ('Ricardo', 'Medina', 'ricardo.medina@email.com', '3344556677'),
    ('Susana', 'Herrera', 'susana.herrera@email.com', '4455667788'),
    ('Mario', 'Domínguez', 'mario.dominguez@email.com', '5566778899'),
    ('Silvia', 'Reyes', 'silvia.reyes@email.com', '6677889900'),
    ('Marcos', 'Fuentes', 'marcos.fuentes@email.com', '7788990011'),
    ('Natalia', 'Cabrera', 'natalia.cabrera@email.com', '8899001122')
]


In [141]:
# OJO: LOS COMANDOS ACÁ SON CASE SENSITIVE
query_clientes = """
INSERT INTO Clientes (Nombre, Apellido, Email, Telefono) VALUES (?,?,?,?)
"""

exec_instruction(query_clientes, many=True, list_rec=clientes)

Executed Succesfully


In [142]:
# @title Datos de las categorías ...  (categorias)
categorias = [
    ('Electrónicos',),
    ('Ropa',),
    ('Alimentos',),
    ('Juguetes',),
    ('Muebles',),
    ('Libros',),
    ('Deportes',),
    ('Jardín',),
    ('Automóviles',),
    ('Belleza',),
    ('Salud',),
    ('Mascotas',),
    ('Oficina',),
    ('Hogar',),
    ('Arte',),
    ('Música',),
    ('Películas',),
    ('Videojuegos',),
    ('Fotografía',),
    ('Computadoras',),
    ('Teléfonos',),
    ('Accesorios',),
    ('Calzado',),
    ('Joyas',),
    ('Relojes',),
    ('Instrumentos Musicales',),
    ('Equipo de Sonido',),
    ('Herramientas',),
    ('Bebés',),
    ('Adultos',),
    ('Otros',)
]


In [143]:
# Tu código acá ...

In [144]:
# @title Datos de productos ... (productos)
productos = [
    ('Televisor', 500.00, 10, 1),
    ('Camisa', 20.00, 50, 2),
    ('Manzana', 1.00, 100, 3),
    ('Pelota', 5.00, 40, 4),
    ('Sofá', 300.00, 5, 5),
    ('Libro', 10.00, 30, 6),
    ('Bicicleta', 100.00, 15, 7),
    ('Maceta', 8.00, 20, 8),
    ('Aceite de motor', 12.00, 25, 9),
    ('Crema facial', 15.00, 60, 10),
    ('Vitamina C', 7.00, 50, 11),
    ('Comida para perro', 20.00, 30, 12),
    ('Silla de oficina', 80.00, 10, 13),
    ('Lámpara', 25.00, 20, 14),
    ('Cuadro', 50.00, 10, 15),
    ('Guitarra', 200.00, 5, 16),
    ('DVD de película', 5.00, 100, 17),
    ('Videojuego', 60.00, 20, 18),
    ('Cámara', 300.00, 8, 19),
    ('Laptop', 800.00, 10, 20),
    ('Smartphone', 700.00, 15, 21),
    ('Mochila', 30.00, 25, 22),
    ('Zapatos', 40.00, 30, 23),
    ('Collar', 100.00, 10, 24),
    ('Reloj', 150.00, 12, 25),
    ('Piano', 1000.00, 2, 26),
    ('Altavoces', 50.00, 20, 27),
    ('Taladro', 70.00, 15, 28),
    ('Pañales', 25.00, 40, 29),
    ('Juguete para adultos', 50.00, 20, 30)
]


In [145]:
# Tu código acá ...

In [146]:
# @title Datos de empleados ... (empleados)
empleados = [
    ('Carlos', 'Rodríguez', 'Vendedor'),
    ('María', 'López', 'Gerente'),
    ('Pedro', 'González', 'Cajero'),
    ('Lucía', 'Martínez', 'Vendedor'),
    ('Fernando', 'García', 'Almacenero'),
    ('Carmen', 'Pérez', 'Vendedor'),
    ('Antonio', 'Sánchez', 'Cajero'),
    ('Miguel', 'Gómez', 'Vendedor'),
    ('Rosa', 'Ruiz', 'Almacenero'),
    ('Francisco', 'Hernández', 'Vendedor'),
    ('Raquel', 'Jiménez', 'Cajero'),
    ('Sergio', 'Díaz', 'Vendedor'),
    ('Isabel', 'Álvarez', 'Almacenero'),
    ('Alejandro', 'Moreno', 'Vendedor'),
    ('Esther', 'Muñoz', 'Cajero'),
    ('Joaquín', 'Romero', 'Vendedor'),
    ('Pilar', 'Alonso', 'Almacenero'),
    ('Andrés', 'Gutiérrez', 'Vendedor'),
    ('Irene', 'Torres', 'Cajero'),
    ('Javier', 'Domínguez', 'Vendedor'),
    ('Teresa', 'Serrano', 'Almacenero'),
    ('Rafael', 'Ramos', 'Vendedor'),
    ('Lorena', 'Blanco', 'Cajero'),
    ('Enrique', 'Medina', 'Vendedor'),
    ('Sonia', 'Castro', 'Almacenero'),
    ('Víctor', 'Ortega', 'Vendedor'),
    ('Laura', 'Cano', 'Cajero'),
    ('Alberto', 'Santos', 'Vendedor'),
    ('Alicia', 'Navarro', 'Almacenero'),
    ('Roberto', 'Gil', 'Vendedor')
]


In [147]:
# Tu código acá ...

In [148]:
# @title datos de ventas ... (ventas)
ventas = [
    ('2023-09-01', 1, 1),
    ('2023-09-02', 2, 1),
    ('2023-09-03', 2, 3),
    ('2023-09-04', 4, 1),
    ('2023-09-05', 2, 5),
    ('2023-09-06', 1, 6),
    ('2023-09-07', 7, 7),
    ('2023-09-08', 8, 8),
    ('2023-09-09', 1, 9),
    ('2023-09-10', 10, 10),
    ('2023-09-11', 11, 1),
    ('2023-09-12', 1, 12),
    ('2023-09-13', 13, 1),
    ('2023-09-14', 1, 14),
    ('2023-09-15', 15, 15),
    ('2023-09-16', 1, 3),
    ('2023-09-17', 17, 17),
    ('2023-09-18', 1, 18),
    ('2023-09-19', 1, 3),
    ('2023-09-20', 20, 20),
    ('2023-09-21', 21, 2),
    ('2023-09-22', 1, 22),
    ('2023-09-23', 23, 23),
    ('2023-09-24', 2, 24),
    ('2023-09-25', 2, 25),
    ('2023-09-26', 2, 26),
    ('2023-09-27', 27, 27),
    ('2023-09-28', 28, 3),
    ('2023-09-29', 2, 4),
    ('2023-09-30', 30, 3)
]


In [149]:
# Tu código acá ...

In [150]:
# @title Datos de detalles de venta ... (detalles_venta)

detalles_venta = [
    (1, 1, 2, 500.00),
    (2, 2, 5, 20.00),
    (3, 3, 10, 1.00),
    (4, 4, 1, 5.00),
    (5, 5, 1, 300.00),
    (6, 6, 3, 10.00),
    (7, 7, 1, 100.00),
    (8, 8, 2, 8.00),
    (9, 9, 1, 12.00),
    (10, 10, 4, 15.00),
    (11, 11, 2, 7.00),
    (12, 12, 1, 20.00),
    (13, 13, 1, 80.00),
    (14, 14, 1, 25.00),
    (15, 15, 1, 50.00),
    (16, 16, 1, 200.00),
    (17, 17, 5, 5.00),
    (18, 18, 1, 60.00),
    (19, 19, 1, 300.00),
    (20, 20, 1, 800.00),
    (21, 21, 1, 700.00),
    (22, 22, 1, 30.00),
    (23, 23, 1, 40.00),
    (24, 24, 1, 100.00),
    (25, 25, 1, 150.00),
    (26, 26, 1, 1000.00),
    (27, 27, 2, 50.00),
    (28, 28, 1, 70.00),
    (29, 29, 2, 25.00),
    (30, 30, 1, 50.00)
]


In [151]:
# Tu código acá ...

## Ahora revisemos como extraer datos de la base de datos

In [153]:
# Llamemos nuestra función con la conexión
conn = get_connection()

In [154]:
cur = conn.cursor()

# El resultado de una ejecución de execute se puede iterar
for row in cur.execute('SELECT * FROM Categorias;'):
    print(row)

# Segurate siempre de cerrar la conexión para que no existan concflictos ... ahora no es tan importante pero más adelante en el curso tomara sentido
conn.close()

(1, 'Electrónicos')
(2, 'Ropa')
(3, 'Alimentos')
(4, 'Juguetes')
(5, 'Muebles')
(6, 'Libros')
(7, 'Deportes')
(8, 'Jardín')
(9, 'Automóviles')
(10, 'Belleza')
(11, 'Salud')
(12, 'Mascotas')
(13, 'Oficina')
(14, 'Hogar')
(15, 'Arte')
(16, 'Música')
(17, 'Películas')
(18, 'Videojuegos')
(19, 'Fotografía')
(20, 'Computadoras')
(21, 'Teléfonos')
(22, 'Accesorios')
(23, 'Calzado')
(24, 'Joyas')
(25, 'Relojes')
(26, 'Instrumentos Musicales')
(27, 'Equipo de Sonido')
(28, 'Herramientas')
(29, 'Bebés')
(30, 'Adultos')
(31, 'Otros')


In [162]:
# Hagamos una query un poco más avanzada: veamos los vendedores que más han facturado para la empresa

query = """
  SELECT
    Empleados.ID_Empleado,
    Empleados.Nombre || ' ' || Empleados.Apellido AS NombreCompleto,
    SUM(DetallesVenta.Cantidad * DetallesVenta.Precio_Unitario) AS TotalVentas
  FROM
    Empleados
  JOIN
    Ventas ON Empleados.ID_Empleado = Ventas.ID_Empleado
  JOIN
    DetallesVenta ON Ventas.ID_Venta = DetallesVenta.ID_Venta
  GROUP BY
    Empleados.ID_Empleado
  ORDER BY
    TotalVentas DESC
"""
conn = get_connection()
cur = conn.cursor()
for row in cur.execute(query):
    print(row)
conn.close()

(1, 'Carlos Rodríguez', 1199.0)
(26, 'Víctor Ortega', 1000.0)
(20, 'Javier Domínguez', 800.0)
(2, 'María López', 700.0)
(3, 'Pedro González', 630.0)
(5, 'Fernando García', 300.0)
(25, 'Sonia Castro', 150.0)
(27, 'Laura Cano', 100.0)
(24, 'Enrique Medina', 100.0)
(7, 'Antonio Sánchez', 100.0)
(18, 'Andrés Gutiérrez', 60.0)
(10, 'Francisco Hernández', 60.0)
(15, 'Esther Muñoz', 50.0)
(4, 'Lucía Martínez', 50.0)
(23, 'Lorena Blanco', 40.0)
(22, 'Rafael Ramos', 30.0)
(6, 'Carmen Pérez', 30.0)
(17, 'Pilar Alonso', 25.0)
(14, 'Alejandro Moreno', 25.0)
(12, 'Sergio Díaz', 20.0)
(8, 'Miguel Gómez', 16.0)
(9, 'Rosa Ruiz', 12.0)


### Ahora respondamos algo...

"¿Cuál es el producto más vendido y cuántas unidades de ese producto se han vendido?"

In [161]:
# "¿Cuál es el producto más vendido y cuántas unidades de ese producto se han vendido?"

# Tu Código acá ...

### Fetch all / Fetch one

In [163]:
conn = get_connection()
cur = conn.cursor()
cur.execute(query)
print(cur.fetchone())
conn.close()

(1, 'Carlos Rodríguez', 1199.0)


In [165]:
conn = get_connection()
cur = conn.cursor()
cur.execute(query)
print(cur.fetchall(), "\n ", type(cur.fetchall()))
conn.close()

[(1, 'Carlos Rodríguez', 1199.0), (26, 'Víctor Ortega', 1000.0), (20, 'Javier Domínguez', 800.0), (2, 'María López', 700.0), (3, 'Pedro González', 630.0), (5, 'Fernando García', 300.0), (25, 'Sonia Castro', 150.0), (27, 'Laura Cano', 100.0), (24, 'Enrique Medina', 100.0), (7, 'Antonio Sánchez', 100.0), (18, 'Andrés Gutiérrez', 60.0), (10, 'Francisco Hernández', 60.0), (15, 'Esther Muñoz', 50.0), (4, 'Lucía Martínez', 50.0), (23, 'Lorena Blanco', 40.0), (22, 'Rafael Ramos', 30.0), (6, 'Carmen Pérez', 30.0), (17, 'Pilar Alonso', 25.0), (14, 'Alejandro Moreno', 25.0), (12, 'Sergio Díaz', 20.0), (8, 'Miguel Gómez', 16.0), (9, 'Rosa Ruiz', 12.0)] 
  <class 'list'>


## Pandas para acceso a la información

In [168]:
import pandas as pd
import sqlite3

# Primero debemos tenter la conección a la base de datos
con = sqlite3.connect("MiTiendaCom.db")
df = pd.read_sql_query("SELECT * from Empleados", con)
con.close()
# Verifiquemos el resultado de la query
print(type(df))
df.head()

<class 'pandas.core.frame.DataFrame'>


ID_Empleado    Nombre   Apellido       Cargo
0            1    Carlos  Rodríguez    Vendedor
1            2     María      López     Gerente
2            3     Pedro   González      Cajero
3            4     Lucía   Martínez    Vendedor
4            5  Fernando     García  Almacenero

In [169]:
import pandas as pd
import sqlite3

# Primero debemos tenter la conección a la base de datos
con = sqlite3.connect("MiTiendaCom.db")
df = pd.read_sql_query(query, con)
con.close()
# Verifiquemos el resultado de la query
print(type(df))
df.head()

<class 'pandas.core.frame.DataFrame'>


ID_Empleado    NombreCompleto  TotalVentas
0            1  Carlos Rodríguez       1199.0
1           26     Víctor Ortega       1000.0
2           20  Javier Domínguez        800.0
3            2       María López        700.0
4            3    Pedro González        630.0

## Guardemos los datos procesados con pandas en una tabla nueva

In [170]:
import pandas as pd
import sqlite3

con = sqlite3.connect("MiTiendaCom.db")

# Cargamos la data en un dataframe
surveys_df = pd.read_sql_query(query, con)


# Escribe el nuevo DataFrame a una nueva tabla en SQLite
surveys_df.to_sql("TopSales", con, if_exists="replace")

con.close()

In [173]:
# Verifiquemos que la nueva tabla exista dentro de la base de datos...

# Tu código acá ...

#### Revisemos ahora como agregar registros nuevos con pandas...


Para esto:

1. Creamos la conexión
2. Traemos con pandas todos los datos de la tabla Empleados
3. Elimanamos la columna de los ID porque son autoincrementales
3. Concatenamos los datos nuevos
4. Sobreescribimos los datos de toda la tabla enviandole el dataframe editado
5. OJO con el idex de pandas, este no lo debemos pasar a la base de datos.


**Vamos a realizar este ejercicio ...**

[Recurso Extra](https://youtu.be/byHcYRpMgI4)

## Hablemos de los ORM's

Un Object-Relational Mapping (ORM) es una técnica que permite interactuar con una base de datos relacional de una manera orientada a objetos. Uno de los ORM más populares para Python es SQLAlchemy. A continuación, te muestro un ejemplo simple de cómo usar SQLAlchemy para modelar una tienda con empleados y productos.

<img src="https://cipsa.net/wp-content/uploads/01-img-Que-es-un-ORM-y-cuando-emplearlo.png.webp"/>


In [174]:
!pip install sqlalchemy

### Ejemplo de ORM en Python

**OJO** Este ejemplo puede que no funcione acá... es solo para ilustrar los conceptos que profundizaremos en una próxima sesión...

In [ ]:
from sqlalchemy import Column, Integer, String, Float, create_engine, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, sessionmaker

Base = declarative_base()

# Definir la clase Empleado, que corresponde a la tabla Empleados en la base de datos
class Empleado(Base):
    __tablename__ = 'empleados'

    id = Column(Integer, primary_key=True)
    nombre = Column(String)
    apellido = Column(String)
    cargo = Column(String)

    def __repr__(self):
        return f"<Empleado(nombre='{self.nombre}', apellido='{self.apellido}', cargo='{self.cargo}')>"

# Definir la clase Producto, que corresponde a la tabla Productos en la base de datos
class Producto(Base):
    __tablename__ = 'productos'

    id = Column(Integer, primary_key=True)
    nombre = Column(String)
    precio = Column(Float)

    def __repr__(self):
        return f"<Producto(nombre='{self.nombre}', precio={self.precio})>"

# Crear una base de datos SQLite en memoria y conectar
engine = create_engine('sqlite:///:memory:')

# Crear las tablas en la base de datos
Base.metadata.create_all(engine)

# Crear una sesión para interactuar con la base de datos
Session = sessionmaker(bind=engine)
session = Session()

# Agregar un nuevo empleado
nuevo_empleado = Empleado(nombre='Juan', apellido='Pérez', cargo='Vendedor')
session.add(nuevo_empleado)
session.commit()

# Agregar un nuevo producto
nuevo_producto = Producto(nombre='Televisor', precio=500.0)
session.add(nuevo_producto)
session.commit()

# Consultar todos los empleados
for empleado in session.query(Empleado).all():
    print(empleado)

# Consultar todos los productos
for producto in session.query(Producto).all():
    print(producto)
